Dissertation
    
# **Notebook 2: Ranking LSOAs Data Preperation**
    
This notebook will prepare data from LSOAs to idenifty ascending LSOAs. The four measures will be:

1. Household median income
2. Median property sale values 
3. Occupation - testing both occupation categories and NS-SEC Class 1
4. Education - Level 4 qualifications and above

***

In [16]:
# Set up notebook 

# Install packages

import pandas as pd
import os
import geopandas as gpd

In [17]:
# Look at working dir

print("The working directory is " + os.getcwd())

The working directory is /home/jovyan/work/OneDrive/UCL/Dissertation/Notebooks Tidy


In [18]:
# Set directories

data_2001 = os.path.join("data", "2001")
data_2001_nomis = os.path.join("data", "2001", "NOMIS")

data_2011 = os.path.join("data", "2011")
data_2011_nomis = os.path.join("data", "2011", "NOMIS")

shapefiles = os.path.join("Shapefiles")

profiles = os.path.join("LSOA Profiles")

In [19]:
if os.path.isdir("Cleaned data") is not True:
    print("Creating 'Cleaned data' directory...")
    os.mkdir("Cleaned data")

In [20]:
# Functions

# Check if merges havebeen successfull

def success(olddataframe, newdataframe):
    print(f"There are " + str(newdataframe.isnull().sum().sum()) + " NaN values in the data frame")
    print(f"These NaN values are located in columns: " + str(newdataframe.columns[newdataframe.isnull().any()].tolist()))
    if len(olddataframe) == len(newdataframe):
        print("Success! The length's are the same")
    else:
        print("Something is wrong!")
        diff = len(newdataframe)-len(olddataframe)
        print(f"{diff} rows are missing")
        
   # Check info of a df

def infodf(dataframe):
    print(f"There are "+str(len(dataframe)) + " rows in the dataframe")
    print(f"There are " + str(dataframe.shape[1]) + " columns in the data frame")
    print(f"The columns of the dataframe are: " + str(dataframe.columns))
    print(f"There are " + str(dataframe.isnull().sum().sum()) + " NaN values in the data frame")
    print(f"These NaN values are located in columns: " + str(dataframe.columns[dataframe.isnull().any()].tolist()))
         
def projection(df1, df2):
    if df1.geometry.crs==df2.geometry.crs:
        print("The projections are the same")
    else:
        print("Need to change the projections")

# 1. 2001 Data

1.1 Read in data frame

In [21]:
# Read in Kent LSOA Profile

census_2001 = pd.read_csv(os.path.join(profiles, "census_2001_profile.csv"))

In [22]:
# Read in spatial data

kent_2001 = gpd.read_file(os.path.join(shapefiles, "LSOA_KENT_2001.shp"))

In [23]:
kent_2001 = kent_2001[["LSOA01CD","geometry"]]

In [24]:
# Make spatial 

kent_2001_gdf = pd.merge(kent_2001, census_2001, how="inner", on="LSOA01CD")

In [25]:
# Filter to required 

kent_2001_gdf = kent_2001_gdf[[
    # Spatial data
    'LSOA01CD', 'geometry', 'OBJECTID', 'LSOA01NM', 'LSOA01NMW', 'Shape__Are', 'Shape__Len', 
    # Population
    'All usual residents',
     # House prices
    'Average_2001',
    # NSSEC
    'All categories: NS-SeC', '1. Higher managerial, administrative and professional occupations',
    # Occupation Categories
     '1. Managers and senior officials',"All categories: Occupation",
    # Education 
    "All categories: Highest level of qualification", "Level 4/5 qualifications",
    # Income
    "Totalweeklyincome(£)"
    # Rural classification
    #"RUC01NM"
]]



In [26]:
# Rename

kent_2001_gdf = kent_2001_gdf.rename(columns={c: c+'_2001' for c in kent_2001_gdf.columns if c not in ['Average_2001']})


In [27]:
# Clean names
kent_2001_gdf.columns = kent_2001_gdf.columns.str.replace(' ','_')


1.2 Property prices

In [28]:
# Read in data

property_2001 = kent_2001_gdf[["LSOA01CD_2001",'Average_2001']]

In [29]:
print("There are " + str(sum([True for idx,row in property_2001.iterrows() if any(row.isnull())])) + " rows missing values")

There are 28 rows missing values


In [31]:
# Read in calculated data - this has mean of neigbs for missing values with orginal values for not missing

calculated_house_prices = pd.read_csv(os.path.join("Data", "Missing_House_Prices_2001_Average.csv"))

In [32]:
property_2001_merge = pd.merge(property_2001, calculated_house_prices, how="left", left_on = "LSOA01CD_2001", right_on="LSOA01CD" )

In [33]:
property_2001_merge.columns

Index(['LSOA01CD_2001', 'Average_2001', 'LSOA01CD', 'Average_2001_Inc_Recalc'], dtype='object')

In [34]:
print("There are " + str(sum([True for idx,row in property_2001_merge.iterrows() if any(property_2001_merge["Average_2001_Inc_Recalc"].isnull())])) + " rows missing values")

There are 935 rows missing values


In [35]:
kent_2001_gdf["Average_2001"] = property_2001_merge["Average_2001_Inc_Recalc"]

1.3 Make data proportional 

In [36]:
print("There are " + str(sum([True for idx,row in kent_2001_gdf.iterrows() if any(row.isnull())])) + " rows missing values")
print("These missing values are in " + str(kent_2001_gdf.columns[kent_2001_gdf.isna().any()].tolist()))

There are 1 rows missing values
These missing values are in ['Average_2001']


1.3 Propotionalise 

In [37]:
# Proportionalise data

kent_2001_gdf["Higher_managerial_administrative_and_professional_occupations_2001_prop"] = kent_2001_gdf["1._Higher_managerial,_administrative_and_professional_occupations_2001"]/kent_2001_gdf["All_categories:_NS-SeC_2001"]
kent_2001_gdf["Managers_and_senior_officials_2001_prop"] = kent_2001_gdf["1._Managers_and_senior_officials_2001"] /kent_2001_gdf["All_categories:_Occupation_2001"] 
kent_2001_gdf["Level_4_qualifications_2001_prop"] = kent_2001_gdf["Level_4/5_qualifications_2001"]  / kent_2001_gdf["All_categories:_Highest_level_of_qualification_2001"]


In [38]:
# Rename

kent_2001_gdf = kent_2001_gdf.rename(columns={"Average_2001":"Mean_house_prices_2001"})

In [39]:
# Calculate the median income 

kent_2001_gdf["Median_income_2001"] = kent_2001_gdf["Totalweeklyincome(£)_2001"] * 52

In [40]:
# Clean for geoconversion export

kent_2001_gdf_clean = kent_2001_gdf[["LSOA01CD_2001", "Mean_house_prices_2001","Higher_managerial_administrative_and_professional_occupations_2001_prop","Managers_and_senior_officials_2001_prop","Level_4_qualifications_2001_prop", "Median_income_2001"]]

In [41]:
# Set data types

kent_2001_gdf_clean[["Mean_house_prices_2001", "Higher_managerial_administrative_and_professional_occupations_2001_prop", "Managers_and_senior_officials_2001_prop", "Level_4_qualifications_2001_prop", "Median_income_2001"]] = kent_2001_gdf_clean[["Mean_house_prices_2001", "Higher_managerial_administrative_and_professional_occupations_2001_prop", "Managers_and_senior_officials_2001_prop", "Level_4_qualifications_2001_prop", "Median_income_2001"]].apply(pd.to_numeric)


/tmp/ipykernel_1292/621514651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kent_2001_gdf_clean[["Mean_house_prices_2001", "Higher_managerial_administrative_and_professional_occupations_2001_prop", "Managers_and_senior_officials_2001_prop", "Level_4_qualifications_2001_prop", "Median_income_2001"]] = kent_2001_gdf_clean[["Mean_house_prices_2001", "Higher_managerial_administrative_and_professional_occupations_2001_prop", "Managers_and_senior_officials_2001_prop", "Level_4_qualifications_2001_prop", "Median_income_2001"]].apply(pd.to_numeric)


In [42]:
# Save

kent_2001_gdf_clean.to_csv(os.path.join("Cleaned data", "kent_2001_before_geoconversion.csv"))

1.4 Read in geoconverted data

In [43]:
edu = pd.read_csv(os.path.join("Cleaned data", "geoconvert", "edugeoconverted.csv"))
cat = pd.read_csv(os.path.join("Cleaned data", "geoconvert", "catgeoconverted.csv"))
nssec = pd.read_csv(os.path.join("Cleaned data", "geoconvert", "nssecgeoconverted.csv"))
house = pd.read_csv(os.path.join("Cleaned data", "geoconvert", "housegeoconverted.csv"))
income = pd.read_csv(os.path.join("Cleaned data", "geoconvert", "incomegeoconverted.csv"))

In [44]:
# Merge

kent_2001_final = pd.merge(edu, cat, on="LSOA11CD", how="inner")
kent_2001_final = pd.merge(kent_2001_final, nssec, on="LSOA11CD", how="inner")
kent_2001_final = pd.merge(kent_2001_final, house, on="LSOA11CD", how="inner")
kent_2001_final = pd.merge(kent_2001_final, income, on="LSOA11CD", how="inner")


In [45]:
# Fix missing LSOA

kent_2001_gdf_clean.loc[934, "Mean_house_prices_2001"]=89369

In [46]:
# Change value
x = kent_2001_gdf_clean[kent_2001_gdf_clean["LSOA01CD_2001"]=="E01024312"]
x = x.rename(columns={"LSOA01CD_2001":"LSOA11CD"})

In [47]:
kent_2001_final = pd.concat([kent_2001_final, x])

In [48]:
if len(edu) == len(cat) == len(nssec) == len(house):
    print("ok")
else:
        print("issue")
        print(len(kent_2001_gdf))
        print(len(cat))

ok


In [49]:
infodf(kent_2001_final)

There are 954 rows in the dataframe
There are 6 columns in the data frame
The columns of the dataframe are: Index(['LSOA11CD', 'Level_4_qualifications_2001_prop',
       'Managers_and_senior_officials_2001_prop',
       'Higher_managerial_administrative_and_professional_occupations_2001_prop',
       'Mean_house_prices_2001', 'Median_income_2001'],
      dtype='object')
There are 0 NaN values in the data frame
These NaN values are located in columns: []


In [50]:
# Clip to actual 902 LSOAs

kent_2011 = gpd.read_file(os.path.join(shapefiles, "LSOA_KENT_2011.shp"))

kent_2001_final = pd.merge(kent_2001_final, kent_2011, on="LSOA11CD", how="right")

In [51]:
# Rename

kent_2001_final = kent_2001_final.rename(columns={"Level_4_qualifications_2001_prop":"Edu_2001_prop", "Managers_and_senior_officials_2001_prop":"NSSEC_level_1_2001_prop", 
                                                  "Higher_managerial_administrative_and_professional_occupations_2001_prop":"Cat_2001_prop"})

# 2. 2011 Data

In [52]:
# Read in Kent LSOA Profile

census_2011 = pd.read_csv(os.path.join(profiles, "census_2011_profile.csv"))

In [53]:
# Read in spatial data

kent_2011 = gpd.read_file(os.path.join(shapefiles, "LSOA_KENT_2011.shp"))

In [54]:
kent_2011 = kent_2011[["LSOA11CD","geometry"]]

In [55]:
# Make spatial 

kent_2011_gdf = pd.merge(kent_2011, census_2011, how="inner", on="LSOA11CD")

In [56]:
# Filter to required 

kent_2011_gdf = kent_2011_gdf[[
    # Spatial data
    'LSOA11CD', 
    # Population
    'All usual residents',
     # House prices
    'Average_2011',
    # Occupations
    'All categories: NS-SeC', '1. Higher managerial, administrative and professional occupations',
    # Occupation Categories
     '1. Managers, directors and senior officials',"All categories: Occupation",
    # Education 
    "All categories: Highest level of qualification", "Highest level of qualification: Level 4 qualifications and above",
    # Income
    "Totalweeklyincome(£)"
    # Rural classification
    #"RUC01NM"
]]


In [57]:
# Clean names

kent_2011_gdf = kent_2011_gdf.rename(columns={c: c+'_2011' for c in kent_2011_gdf.columns if c not in ['Average_2011', "LSOA11CD"]})

In [58]:
kent_2011_gdf.columns = kent_2011_gdf.columns.str.replace(' ','_')

In [59]:
print("There are " + str(sum([True for idx,row in kent_2011_gdf.iterrows() if any(row.isnull())])) + " rows missing values")
print("These missing values are in " + str(kent_2011_gdf.columns[kent_2011_gdf.isna().any()].tolist()))

There are 0 rows missing values
These missing values are in []


In [60]:
# Proportionalise data

kent_2011_gdf["NSSEC_level_1_2011_prop"] = kent_2011_gdf["1._Higher_managerial,_administrative_and_professional_occupations_2011"]/kent_2011_gdf["All_categories:_NS-SeC_2011"]
kent_2011_gdf["Cat_2011_prop"] = kent_2011_gdf["1._Managers,_directors_and_senior_officials_2011"] /kent_2011_gdf["All_categories:_Occupation_2011"] 
kent_2011_gdf["Edu_2011_prop"] = kent_2011_gdf["Highest_level_of_qualification:_Level_4_qualifications_and_above_2011"]  / kent_2011_gdf["All_categories:_Highest_level_of_qualification_2011"]
kent_2011_gdf["Median_income_2011"] = kent_2011_gdf["Totalweeklyincome(£)_2011"] * 52
kent_2011_gdf = kent_2011_gdf.rename(columns={"Average_2011":"Mean_house_prices_2011"})

In [61]:
kent_2011_gdf.replace(',','', regex=True, inplace=True)

In [62]:
# Replace DIV issue 

kent_2011_gdf.loc[401, "Mean_house_prices_2011"]=196060

In [63]:
kent_2011_gdf[['Median_income_2011', 'NSSEC_level_1_2011_prop', 'Cat_2011_prop','Edu_2011_prop', "Mean_house_prices_2011"]] = kent_2011_gdf[['Median_income_2011', 'NSSEC_level_1_2011_prop', 'Cat_2011_prop',       'Edu_2011_prop', "Mean_house_prices_2011"]].apply(pd.to_numeric)

In [64]:
kent_2011_gdf_clean = kent_2011_gdf[["LSOA11CD", 'Median_income_2011', 'NSSEC_level_1_2011_prop', 'Cat_2011_prop','Edu_2011_prop', "Mean_house_prices_2011"]]

In [65]:
kent_2011_final = kent_2011_gdf_clean

# 4. Merge dataframes

In [66]:
df = pd.merge(kent_2001_final, kent_2011_final, how="right", on="LSOA11CD")

In [67]:
# Put into order

df = df[["LSOA11CD",
     
     # Income     
    "Median_income_2001", "Median_income_2011",
       
     # House prices     
    "Mean_house_prices_2001", "Mean_house_prices_2011",
       
     # Education
    "Edu_2001_prop", "Edu_2011_prop",
     
     # NSSEC
     "NSSEC_level_1_2001_prop","NSSEC_level_1_2011_prop",
       
     # Job catagories 
     "Cat_2001_prop", "Cat_2011_prop"
     ]]

In [68]:
infodf(df)

There are 902 rows in the dataframe
There are 11 columns in the data frame
The columns of the dataframe are: Index(['LSOA11CD', 'Median_income_2001', 'Median_income_2011',
       'Mean_house_prices_2001', 'Mean_house_prices_2011', 'Edu_2001_prop',
       'Edu_2011_prop', 'NSSEC_level_1_2001_prop', 'NSSEC_level_1_2011_prop',
       'Cat_2001_prop', 'Cat_2011_prop'],
      dtype='object')
There are 0 NaN values in the data frame
These NaN values are located in columns: []


In [69]:
# Lets check the df

print("We have " + str(len(kent_2001_final)) + " LSOAs in final 2001 df")
print("We have " + str(len(kent_2011_final)) + " LSOAs in final 2011 df")
print("We have " + str(len(df)) + " LSOAs in final df")

if len(df)==len(kent_2011):
    print("All LSOAs are there")
else:
    print("We are missing " + str(len(kent_2011)- len(df)) + " LSOA(s)")
print("But there is a discrepancy")

We have 902 LSOAs in final 2001 df
We have 902 LSOAs in final 2011 df
We have 902 LSOAs in final df
All LSOAs are there
But there is a discrepancy


In [70]:
success(df, kent_2001_final)
success(df, kent_2011_final)

There are 0 NaN values in the data frame
These NaN values are located in columns: []
Success! The length's are the same
There are 0 NaN values in the data frame
These NaN values are located in columns: []
Success! The length's are the same


In [71]:
df.to_csv(os.path.join("Cleaned data", "ranking_vars.csv"))